In [ ]:
!pip install transformers datasets evaluate accelerate

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json(f'/content/datafinal.json')

In [ ]:
df.head()

,ID,Text,ContainsCode,CodeList
0,1,Test-driven development (TDD) is a software de...,,
1,2,Software development is an exciting field that...,False,
2,3,The development process often starts with a co...,,
3,4,Another important aspect of software developme...,True,git commit -m 'Initial commit'
4,5,"In the world of software development, language...",False,


In [ ]:
df.ContainsCode.value_counts()

False    845
True     532
         531
Name: ContainsCode, dtype: int64

In [ ]:
nonCodeText = df[df['ContainsCode'] == True]['Text'].tolist()

In [ ]:
temp = 0
for row in df.iterrows():
  text = row[1]['Text']
  if temp < len(text.split(' ')):
    temp = len(text.split(' '))

temp

266

In [ ]:
from datasets import Dataset

cleaned_df = df[df['ContainsCode'] != ""]
cleaned_df['label'] = cleaned_df['ContainsCode'].map(lambda x: 1 if x else 0)
dataset = Dataset.from_pandas(cleaned_df)

<ipython-input-7-02c17f103b54>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['label'] = cleaned_df['ContainsCode'].map(lambda x: 1 if x else 0)


In [ ]:
dataset

Dataset({
    features: ['ID', 'Text', 'ContainsCode', 'CodeList', 'label', '__index_level_0__'],
    num_rows: 1377
})

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["Text"], truncation=True)

tokenized_imdb = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1377 [00:00<?, ? examples/s]

In [ ]:
tokenized_imdb

Dataset({
    features: ['ID', 'Text', 'ContainsCode', 'CodeList', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 1377
})

In [ ]:
from datasets import DatasetDict

# 90% train, 10% test + validation
train_testvalid = tokenized_imdb.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [ ]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'Text', 'ContainsCode', 'CodeList', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1239
    })
    test: Dataset({
        features: ['ID', 'Text', 'ContainsCode', 'CodeList', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 69
    })
    valid: Dataset({
        features: ['ID', 'Text', 'ContainsCode', 'CodeList', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 69
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_valid_dataset["train"],
    eval_dataset=train_test_valid_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000586,1.000000
2,No log,0.000379,1.000000


TrainOutput(global_step=156, training_loss=0.008016201166006235, metrics={'train_runtime': 49.4988, 'train_samples_per_second': 50.062, 'train_steps_per_second': 3.152, 'total_flos': 113337502192668.0, 'train_loss': 0.008016201166006235, 'epoch': 2.0})

In [ ]:
import torch

text = "It was raining today - bad"
tokenizer = AutoTokenizer.from_pretrained("/content/my_awesome_model/checkpoint-156")
inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForSequenceClassification.from_pretrained("/content/my_awesome_model/checkpoint-156")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'False'

In [ ]:
cleaned_df['Text'].tolist()[1]

"Another important aspect of software development is version control. Version control systems such as Git allow developers to track changes to their code, collaborate with others, and revert to previous versions of their code if necessary. It's an essential tool for any software developer.\nOne of the most important commands in Git is the commit command:\ngit commit -m 'Initial commit'\nThis command will create a new commit with the message 'Initial commit', representing a snapshot of the code at that point in time."

In [ ]:
cleaned_df

,ID,Text,ContainsCode,CodeList,label
1,2,Software development is an exciting field that...,False,,0
3,4,Another important aspect of software developme...,True,git commit -m 'Initial commit',1
4,5,"In the world of software development, language...",False,,0
7,8,Software development is an intricate process t...,False,,0
8,9,"In this modern era, software development has i...",True,public class HelloWorld { public static void m...,1
...,...,...,...,...,...
1901,1902,Software development is more than just coding....,False,,0
1902,1903,One of the most popular languages for software...,True,public class HelloWorld {\npublic static void ...,1
1903,1904,"Software development, an integral part of any ...",True,List<String> names = new ArrayList<String>();,1
1904,1905,"In the world of software development, one of t...",False,,0


434